# ETL - Fase 4: Consolidamos los df para alimentar las consultas
En esta seccion, consolidamos lo df generados en las fases anteriores de ETL. De igual manera, creamos dos subconjuntos de datos para alimentar los principales dos tipos de consulta: las funciones generales y el sistema de recomendacion (modelo ML).

In [1]:
import os
import pandas as pd

In [2]:
data_movies_subset = pd.read_csv(
    os.path.join("2_pipeline","data_movies_subset_limpia.csv"),index_col=0).convert_dtypes() # output paso 01
data_movies_normalizada = pd.read_csv(
    os.path.join("2_pipeline","data_movies_normalizada.csv"),index_col=0).convert_dtypes() # output paso 02
data_credits_normalizada = pd.read_csv(
    os.path.join("2_pipeline","credits_normalizada.csv"),index_col=0).convert_dtypes() # output paso 03

In [3]:
data_movies_subset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45465
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45346 non-null  Int64  
 1   pelicula_id        45346 non-null  Int64  
 2   original_language  45335 non-null  string 
 3   overview           44405 non-null  string 
 4   popularity         45346 non-null  Float64
 5   release_date       45346 non-null  string 
 6   revenue            45346 non-null  Int64  
 7   runtime            45100 non-null  Int64  
 8   title              45346 non-null  string 
 9   vote_average       45346 non-null  Float64
 10  vote_count         45346 non-null  Int64  
 11  release_year       45346 non-null  Int64  
 12  return             45346 non-null  Float64
dtypes: Float64(3), Int64(6), string(4)
memory usage: 5.2 MB


In [4]:
data_movies_normalizada.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 55897 entries, 0 to 55896
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   pelicula_id               55897 non-null  Int64 
 1   franquicia_id             55897 non-null  Int64 
 2   franquicia                55897 non-null  string
 3   productora                53879 non-null  string
 4   productora_id             53879 non-null  Int64 
 5   genres_id                 55754 non-null  Int64 
 6   genres                    55754 non-null  string
 7   pais_isocode              55285 non-null  string
 8   pais_name                 55285 non-null  string
 9   spoken_languages_isocode  55728 non-null  string
 10  spoken_languages_name     55355 non-null  string
dtypes: Int64(4), string(7)
memory usage: 5.3 MB


In [5]:
data_credits_normalizada.info(verbose=True)
data_credits_normalizada.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 106173 entries, 0 to 106172
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   pelicula_id           106173 non-null  Int64 
 1   prtgnst_name          106173 non-null  string
 2   prtgnst_gender_strng  106173 non-null  string
 3   prtgnst_nivel         106173 non-null  Int64 
 4   director              106138 non-null  string
 5   executive_producer    38468 non-null   string
dtypes: Int64(2), string(4)
memory usage: 5.9 MB


,pelicula_id,prtgnst_name,prtgnst_gender_strng,prtgnst_nivel,director,executive_producer
0,862,Tom Hanks,hombre,1,John Lasseter,Ed Catmull
1,862,Tom Hanks,hombre,1,John Lasseter,Steve Jobs
2,862,Tim Allen,hombre,2,John Lasseter,Ed Catmull
3,862,Tim Allen,hombre,2,John Lasseter,Steve Jobs
4,8844,Robin Williams,hombre,1,Joe Johnston,Larry J. Franco
5,8844,Robin Williams,hombre,1,Joe Johnston,Ted Field
6,8844,Robin Williams,hombre,1,Joe Johnston,Robert W. Cort
7,8844,Jonathan Hyde,hombre,2,Joe Johnston,Larry J. Franco
8,8844,Jonathan Hyde,hombre,2,Joe Johnston,Ted Field
9,8844,Jonathan Hyde,hombre,2,Joe Johnston,Robert W. Cort


## Analisis de disponibilidad de datos


En esta seccion examinamos la disponibilidad de datos para nuestro universo de casos: peliculas. Conocer la disponibilidad de datos es indispensable por dos razones:

1. Determina la naturaleza de los joins: Un paso previo a la union de dfs es determinar cual de los dfs contiene la mayor cantidad de informacion disponible para nuestro universo de casos: peliculas. Esto determina la naturaleza del join; es decir, si hacemos un inner o un left join. En caso que un df contenga informacion sobre un mayor numero de casos que el otro, hacemos un left join con el fin garantizar el mayor numero de registros posible.

2. Afecta el desempeño de las consultas: diferencias en las disponibilidad de informacion puede afectar el desempeno de unas consultas con respecto a otras; es decir, mientras que algunas consultas pueden gozar de una gran disponibilidad de informacion, otras pueden verse seriamente afectadas por informacion limitada. Conocer esta ventajas y limitaciones nos permite anticipar futuros problemas en el desempeno de las consultas.

### Descripcion de disponibilidad de informacion

In [6]:
# evaluamos cual df contiene informacion para el mayor numero de casos.
movies_subset_peliculas=data_movies_subset['pelicula_id'].nunique()
movies_normalizada_peliculas=data_movies_normalizada['pelicula_id'].nunique()
credits_normalizada_peliculas=data_credits_normalizada['pelicula_id'].nunique()
print("Universo de pelicula unicas, segun df de referencia:",movies_subset_peliculas)
print("Universo de pelicula unicas, segun movies_normalizada:",movies_normalizada_peliculas)
print("Universo de pelicula unicas, segun credits_normalizada:",credits_normalizada_peliculas)
print("Porcentaje de valores nulos en columnas de movies_normalizada del total de registros (df referencia):",
    round((1-(movies_normalizada_peliculas/movies_subset_peliculas))*100,2),"%")
print("Porcentaje de valores nulos en columnas de credits_normalizada del total de registros (df referencia):",
    round((1-(credits_normalizada_peliculas/movies_subset_peliculas))*100,2),"%")

Universo de pelicula unicas, segun df de referencia: 45346
Universo de pelicula unicas, segun movies_normalizada: 4487
Universo de pelicula unicas, segun credits_normalizada: 42317
Porcentaje de valores nulos en columnas de movies_normalizada del total de registros (df referencia): 90.1 %
Porcentaje de valores nulos en columnas de credits_normalizada del total de registros (df referencia): 6.68 %


Con base en lo anterior, la base de datos final tendria la siguientes caracteristicas:
Todos las peliculas tendria valores nulos en alguna columna.
Las columnas con mayor numero de valores nulos serian las columnas que corresponden al df `data_movies_normalizada`.

### Implicaciones de disponibilidad de datos para las consultas

Las funciones que mas se afectarian por esta baja disponibilidad de datos seria las siguientes consultas:

- CUALES?????

Lo anterior debe orientar nuestro diseno del modelo de ML. Un modelo que haga uso de las columnas que corresponden a data_movies_normalizada (i.e., franquicia, productora, el genero de la pelicula (genres), e idiomas hablados (spoken_languages_name)), estaria desaprovechando la disponbilidad de datos, puesto que solo tendria informacion para el ~10% de universo de casos en nuestra base de dato final.

Por tal motivo, lo mejor seria hacer uso la informacion contenida en data_movies_subset y en data_credits_normalizada al diseñar el modelo de ML.

## Generacion de dfs para alimentar las consultas

In [7]:
# exportamos el df final
data_mvp_final_normalizada = pd.merge(
    pd.merge(data_movies_subset, # tomamos como principal referencia el df con el mayor numero de registros
    data_movies_normalizada,
    how='left',
    on=['pelicula_id']),
    data_credits_normalizada, # tomamos como ultima referencia el df con el menor numero de registros
    how='left',
    on=['pelicula_id'])
data_mvp_final_normalizada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310040 entries, 0 to 310039
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   budget                    310040 non-null  Int64  
 1   pelicula_id               310040 non-null  Int64  
 2   original_language         310024 non-null  string 
 3   overview                  308284 non-null  string 
 4   popularity                310040 non-null  Float64
 5   release_date              310040 non-null  string 
 6   revenue                   310040 non-null  Int64  
 7   runtime                   309672 non-null  Int64  
 8   title                     310040 non-null  string 
 9   vote_average              310040 non-null  Float64
 10  vote_count                310040 non-null  Int64  
 11  release_year              310040 non-null  Int64  
 12  return                    310040 non-null  Float64
 13  franquicia_id             214036 non-null  I

Separamos el df final en dos subconjuntos:
1. df para alimentar las funciones
2. df para alimentar el modelo ml y el sistema de recomendacion.

In [8]:
# primer subconjunto: df para alimentar las funciones
columns_funciones=[
    'pelicula_id', # todas
    'title', # varias
    'original_language', # def peliculas_idioma( Idioma: str )
    'runtime', # def peliculas_duracion( Pelicula: str )
    'release_date', # def peliculas_duracion( Pelicula: str )
    'franquicia', # def franquicia( Franquicia: str )
    'budget', # def franquicia( Franquicia: str )
    'revenue', # def franquicia( Franquicia: str )
    'pais_name', # def peliculas_pais( Pais: str )
    'productora', # productoras_exitosas( Productora: str ),
    'director', # get_director( nombre_director )
    'return', # get_director( nombre_director )
    'release_year' # get_director( nombre_director )
    ]

In [9]:
data_mvp_funciones=data_mvp_final_normalizada.loc[:,columns_funciones].drop_duplicates()
print("Universo de peliculas unicas en el df que alimenta las funciones:",
      data_mvp_funciones['pelicula_id'].nunique())
data_mvp_funciones.to_csv(os.path.join("3_output","data_mvp_final_funciones.csv"))
data_mvp_funciones.info()

Universo de peliculas unicas en el df que alimenta las funciones: 45346
<class 'pandas.core.frame.DataFrame'>
Index: 59545 entries, 0 to 310039
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pelicula_id        59545 non-null  Int64  
 1   title              59545 non-null  string 
 2   original_language  59531 non-null  string 
 3   runtime            59283 non-null  Int64  
 4   release_date       59545 non-null  string 
 5   franquicia         15315 non-null  string 
 6   budget             59545 non-null  Int64  
 7   revenue            59545 non-null  Int64  
 8   pais_name          15020 non-null  string 
 9   productora         14581 non-null  string 
 10  director           56423 non-null  string 
 11  return             59545 non-null  Float64
 12  release_year       59545 non-null  Int64  
dtypes: Float64(1), Int64(5), string(7)
memory usage: 6.7 MB


In [10]:
# segundo subconjunto: df para alimentar el modelo de ML
columns_ml=[
    'pelicula_id', # todas
    # CUALEES?
    ]

In [11]:
# df final para modelo ml
data_mvp_ml=data_mvp_final_normalizada.loc[:,columns_ml].drop_duplicates()
print("Universo de peliculas unicas en el df que alimenta el modelo ml:",
      data_mvp_funciones['pelicula_id'].nunique())
#data_mvp_funciones.to_csv(os.path.join("3_output","data_mvp_final_ml.csv"))
#data_mvp_funciones.info()